In [2]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
import numpy as np

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("samuelcortinhas/muffin-vs-chihuahua-image-classification")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\jessi\.cache\kagglehub\datasets\samuelcortinhas\muffin-vs-chihuahua-image-classification\versions\2


In [4]:

# DATASET DIRECTORY CONFIGURATION
train_dir = os.path.join(path, "train")
test_dir = os.path.join(path, "test")

print("Train directory:", train_dir)
print("Test directory:", test_dir)

Train directory: C:\Users\jessi\.cache\kagglehub\datasets\samuelcortinhas\muffin-vs-chihuahua-image-classification\versions\2\train
Test directory: C:\Users\jessi\.cache\kagglehub\datasets\samuelcortinhas\muffin-vs-chihuahua-image-classification\versions\2\test


In [5]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (224, 224) # changed because chosen architecture prefers this image size than 128x128
BATCH_SIZE = 32

In [6]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [7]:
# VGG16 Model Architecture

#Loading pretrained VGG16 and removing top layers
base_model = VGG16(
    weights='imagenet', 
    include_top = False, 
    input_shape = (224,224, 3))
base_model.trainable = False #freezing base layers

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    base_model, 
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation = 'sigmoid')
])

In [8]:
model.compile(
    optimizer = optimizer,
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [9]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 685s 6s/step - accuracy: 0.9073 - loss: 0.3023 - val_accuracy: 0.9683 - val_loss: 0.0876
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 683s 6s/step - accuracy: 0.9644 - loss: 0.1022 - val_accuracy: 0.9725 - val_loss: 0.0809
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 728s 6s/step - accuracy: 0.9681 - loss: 0.0861 - val_accuracy: 0.9661 - val_loss: 0.0981
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1048s 9s/step - accuracy: 0.9696 - loss: 0.0779 - val_accuracy: 0.9640 - val_loss: 0.0849
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 698s 6s/step - accuracy: 0.9784 - loss: 0.0581 - val_accuracy: 0.9746 - val_loss: 0.0735
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 685s 6s/step - accuracy: 0.9731 - loss: 0.0767 - val_accuracy: 0.9746 - val_loss: 0.0753
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 672s 6s/step - accuracy: 0.9741 - loss: 0.0687 - val_accuracy: 0.9524 - val_loss: 0.1286
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 739s 6s/step - accuracy: 0.9741 - loss: 0.0628 - val_acc

In [10]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 172s 5s/step - accuracy: 0.9831 - loss: 0.0584
Test Accuracy: 0.9831081032752991


In [17]:
# SAVE THE MODEL
model.save('muffin_vs_chihuahua_cnn.h5')

In [18]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)[0,0]

    if pred >= 0.5:
        label = "Muffin"
        confidence = pred
    else:
        label = "Chihuahua"
        confidence = 1 - pred  # flip for correct confidence
    
    print(f"Prediction: {label} (confidence: {confidence:.2f})")


In [19]:
predict_image("muffin vs. chihuahua predict_images/img_1.jpg")
predict_image("muffin vs. chihuahua predict_images/img_2.jpg")
predict_image("muffin vs. chihuahua predict_images/img_3.jpg")
predict_image("muffin vs. chihuahua predict_images/img_4.jpg")
predict_image("muffin vs. chihuahua predict_images/img_5.jpg")
predict_image("muffin vs. chihuahua predict_images/img_6.jpg")
predict_image("muffin vs. chihuahua predict_images/img_7.jpg")
predict_image("muffin vs. chihuahua predict_images/img_8.jpg")
predict_image("muffin vs. chihuahua predict_images/img_9.jpg")
predict_image("muffin vs. chihuahua predict_images/img_10.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
Prediction: Muffin (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
Prediction: Chihuahua (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
Prediction: Chihuahua (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
Prediction: Chihuahua (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
Prediction: Chihuahua (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 897ms/step
Prediction: Muffin (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
Prediction: Muffin (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
Prediction: Chihuahua (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
Prediction: Muffin (confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
Prediction: Muffin (confidence: 1.00)


In [20]:
#double check ko lang basi baliskad XD
train_generator.class_indices

{'chihuahua': 0, 'muffin': 1}